# NFL Route Classifier – Week 9 Ensemble Report

This notebook trains (optionally) and visualizes the latest ensemble model results (BiGRU + Transformer) on the strict Week 9 holdout.

- Training data: Weeks 1–8
- Test data: Week 9
- Features: tracking-derived sequence features + QB/LOS alignment + cut descriptors
- Models: BiGRU+Attention and small Transformer; logits averaged
- Optional pairwise fixers (IN↔OUT, POST↔CORNER) using QB/center cut features



In [ ]:
# Setup
import os, sys, json
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
ROOT = Path('.')



In [ ]:
# Run ensemble training/evaluation (will print metrics)
!python3 train_ensemble_route_week9.py


In [ ]:
# Load confusion matrix and metrics for visualization
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

counts_csv = 'confusion_matrix_week9_counts_ensemble_fixers.csv'
df_counts = pd.read_csv(counts_csv, index_col=0)

# Heatmap - counts
plt.figure(figsize=(10,8))
sns.heatmap(df_counts, annot=False, cmap='Blues')
plt.title('Week 9 Confusion Matrix (Ensemble+Fixers, Counts)')
plt.xlabel('Predicted'); plt.ylabel('True')
plt.tight_layout()
plt.show()


In [ ]:
# Row-normalized heatmap (build from counts)
cm = df_counts.values
row_sums = cm.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
cm_norm = (cm / row_sums).astype(float)
df_norm = pd.DataFrame(cm_norm, index=df_counts.index, columns=df_counts.columns)

plt.figure(figsize=(10,8))
sns.heatmap(df_norm, annot=False, cmap='Greens', vmin=0, vmax=1)
plt.title('Week 9 Confusion Matrix (Ensemble+Fixers, Row-normalized)')
plt.xlabel('Predicted'); plt.ylabel('True')
plt.tight_layout()
plt.show()


In [ ]:
# Per-class F1 from confusion matrix approximation (requires true metrics; for quick view use recall)
recall = np.diag(cm) / row_sums.ravel()
plt.figure(figsize=(8,6))
order = np.argsort(recall)
plt.barh(df_counts.index[order], recall[order])
for i, v in enumerate(recall[order]):
    plt.text(v + 0.005, i, f"{v:.2f}", va='center', fontsize=8)
plt.xlim(0,1)
plt.title('Per-class Recall (Week 9) – Ensemble+Fixers')
plt.xlabel('Recall')
plt.tight_layout()
plt.show()


## Notes
- You can re-run the training cell to regenerate metrics and confusion matrix CSVs.
- For a richer report, also run `plot_week9_report.py` for the BiGRU-only results or adapt it for the ensemble.
- To export predictions per sample, extend `train_ensemble_route_week9.py` to dump predicted labels and probabilities for Week 9.
